# Using LIME to extract relevant features for PDF classifier.

### PDF-data and trained model is shared in the ./data folder. 
#### We utilize [captum](https://captum.ai/) for computing explanations using LIME and [quantus](https://quantus.readthedocs.io/en/latest/) to quantitatively evaluate the explanations. 

In [131]:
import warnings
warnings.filterwarnings('ignore')

In [132]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.utils.data as utils
import torch.utils.data as td
torch.manual_seed(1234)
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf

In [173]:
path_to_csv = "data/contagio-all.csv"

In [134]:
# Load datasets
df = pd.read_csv(path_to_csv)

In [135]:
df =df.drop(['filename'], axis=1)

In [136]:
df.head()

,class,author_dot,author_lc,author_len,author_mismatch,author_num,author_oth,author_uc,box_nonother_types,box_other_only,...,subject_oth,subject_uc,title_dot,title_lc,title_len,title_mismatch,title_num,title_oth,title_uc,version
0,0,0,0,0,0,0,0,0,18,0,...,0,0,0,60,90,1,4,13,13,4
1,0,0,0,11,1,0,3,8,6,0,...,8,6,0,3,17,1,5,5,4,7
2,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,4
3,0,0,9,12,1,0,1,2,20,0,...,0,0,0,0,16,8,0,2,14,4
4,0,3,27,48,1,0,13,8,3,0,...,3,2,3,14,28,1,0,9,5,3


In [137]:

# Pandas dataframes to numpy arrays
X = df.drop(["class"], axis=1).values
Y = df["class"].values


In [138]:
from sklearn.preprocessing import normalize

binary_encoding = False 
if binary_encoding: 
    X[np.where(data!= 0)] = 1
else:
    X = normalize(X, 'max', axis=0)

In [139]:
train_features, test_features, train_labels, test_labels = train_test_split(
    X, Y, test_size=0.3
)

In [140]:
train_features.shape

(6999, 135)

In [141]:
test_features.shape

(3000, 135)

In [142]:
class NNModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear1 = nn.Linear(135, 135)
        self.sigmoid1 = nn.Sigmoid()
        self.linear2 = nn.Linear(135, 100)
        self.sigmoid2 = nn.Sigmoid()
        self.linear3 = nn.Linear(100, 8)
        self.sigmoid3 = nn.Sigmoid()
        self.linear4 = nn.Linear(8, 2)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        lin1_out = self.linear1(x)
        sigmoid_out1 = self.sigmoid1(lin1_out)
        sigmoid_out2 = self.sigmoid2(self.linear2(sigmoid_out1))
        sigmoid_out3 = self.sigmoid3(self.linear3(sigmoid_out2))
        return self.softmax(self.linear4(sigmoid_out3))

In [143]:
net = NNModel()

criterion = nn.CrossEntropyLoss()
num_epochs = 300

optimizer = torch.optim.Adam(net.parameters(), lr=0.001)
input_tensor = torch.from_numpy(train_features).type(torch.FloatTensor)
label_tensor = torch.from_numpy(train_labels)


In [14]:
# Use this to train the model 

'''
for epoch in range(num_epochs):
    output = net(input_tensor)
    loss = criterion(output, label_tensor)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if epoch % 20 == 0:
        print("Epoch {}/{} => Loss: {:.2f}".format(epoch + 1, num_epochs, loss.item())) 

'''

'\nfor epoch in range(num_epochs):\n    output = net(input_tensor)\n    loss = criterion(output, label_tensor)\n    optimizer.zero_grad()\n    loss.backward()\n    optimizer.step()\n    if epoch % 20 == 0:\n        print("Epoch {}/{} => Loss: {:.2f}".format(epoch + 1, num_epochs, loss.item())) \n\n'

In [144]:
## load the trained model
path = "PDFClassifierFinal.pth"
net = NNModel()
net.load_state_dict(torch.load(path))
net.eval()

NNModel(
  (linear1): Linear(in_features=135, out_features=135, bias=True)
  (sigmoid1): Sigmoid()
  (linear2): Linear(in_features=135, out_features=100, bias=True)
  (sigmoid2): Sigmoid()
  (linear3): Linear(in_features=100, out_features=8, bias=True)
  (sigmoid3): Sigmoid()
  (linear4): Linear(in_features=8, out_features=2, bias=True)
  (softmax): Softmax(dim=1)
)

In [145]:
out_probs = net(input_tensor).detach().numpy()
out_classes = np.argmax(out_probs, axis=1)
print("Train Accuracy:", sum(out_classes == train_labels) / len(train_labels))

Train Accuracy: 0.9781397342477497


In [146]:
test_input_tensor = torch.from_numpy(test_features).type(torch.FloatTensor)
out_probs = net(test_input_tensor).detach().numpy()
out_classes = np.argmax(out_probs, axis=1)
print("Test Accuracy:", sum(out_classes == test_labels) / len(test_labels))

Test Accuracy: 0.977


In [147]:
#generate test_label_tensor to be used in quantitative evaluation
test_labels_tensor =  torch.from_numpy(test_labels)

# Generate explanations and then metrics

In [148]:
from captum.attr import LimeBase

In [149]:
from captum._utils.models.linear_model import SkLearnLinearModel

In [150]:
#functions for LIME; taken from Captum 

def similarity_kernel(original_input, perturbed_input, perturbed_interpretable_input,**kwargs):
    kernel_width = kwargs["kernel_width"]
    l2_dist = torch.norm(original_input - perturbed_input)
    return torch.exp(- (l2_dist**2) / (kernel_width**2))

def perturb_func(original_input,**kwargs):
    return original_input + torch.randn_like(original_input)

def to_interp_transform(curr_sample, original_inp,**kwargs):
    return curr_sample

In [151]:
lime_a = LimeBase(net,
                         SkLearnLinearModel("linear_model.Ridge"),
                         similarity_func=similarity_kernel,
                         perturb_func=perturb_func,
                         perturb_interpretable_space=False,
                         from_interp_rep_transform=None,
                         to_interp_rep_transform=to_interp_transform)

In [34]:
test_feature = test_input_tensor[1:800]
test_label = test_labels_tensor[1:800]
new_test_label = []
new_test_feature = []
for i in range(len(test_label)):
    if (test_label[i] == 1):
        new_test_label.append(test_label[i])
        new_test_feature.append(test_feature[i])

In [35]:
new_test_feature_tensor = torch.stack(new_test_feature)
new_test_label_tensor = torch.stack(new_test_label)

In [36]:
new_test_label_tensor.shape

torch.Size([405])

## Get feature attributions

In [47]:
lime_attribution = []
for i in range(400):
    lime_attr = lime_a.attribute(new_test_feature_tensor[i:i+1], target = new_test_label_tensor[i:i+1], kernel_width=1.1)
    lime_attribution.append(lime_attr)

In [48]:
final_values=[]
for lime_attr in lime_attribution:
    lime_attr_test_sum = lime_attr.detach().numpy().sum(0)
    lime_attr_test_norm_sum = lime_attr_test_sum / np.linalg.norm(lime_attr_test_sum, ord=1)
    final_values.append(lime_attr_test_norm_sum)

In [49]:
lime_fina_attr=(np.mean(final_values, axis=0))

In [50]:
path_to_csv = "pdf-data/data/contagio-all.csv"
data = pd.read_csv(path_to_csv, dtype=str, delimiter=',',)
data = data.drop(['filename'], axis=1)
data = data.drop(['class'], axis=1)
feature_names = data.columns
x_axis_data = np.arange(test_features.shape[1])

In [51]:
k=10
top_15_ind = np.argsort(lime_fina_attr)[-k:]
new_array = lime_fina_attr[top_15_ind]
top_15_labels = [x_axis_data_labels[i] for i in top_15_ind]
top_15_labels.reverse()
top_15_labels

['count_js',
 'len_obj_min',
 'keywords_uc',
 'pdfid1_oth',
 'pdfid0_oth',
 'pos_acroform_avg',
 'keywords_lc',
 'count_js_obs',
 'count_action',
 'count_javascript']

# Compute metrics

In [153]:
import quantus

# Generate explanation wrapper 

In [154]:
lime_attr = LimeBase(net,
                         SkLearnLinearModel("linear_model.Ridge"),
                         similarity_func=similarity_kernel,
                         perturb_func=perturb_func,
                         perturb_interpretable_space=False,
                         from_interp_rep_transform=None,
                         to_interp_rep_transform=to_interp_transform)


In [155]:
def explainer_wrapper(**kwargs):
    """Wrapper for explainer functions."""
    if kwargs["method"] == "Lime_explain":
        return lime_explainer(**kwargs)
    else: 
        raise ValueError("Pick an explaination function that exists.")

In [156]:
def lime_explainer(model, inputs, targets, lime_attr):
    
    inputs = torch.from_numpy(inputs).type(torch.FloatTensor)
    targets = torch.from_numpy(targets) 
    lime_attr1 = lime_attr.attribute(inputs, target = targets, kernel_width=1.1)
    lime_attr2 = lime_attr1.cpu().data
    if lime_attr2.requires_grad:
        return lime_attr2.cpu().detach().numpy()
    else: 
        return lime_attr2.cpu().numpy()
    

In [157]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [158]:
import gc

In [159]:
scores_modelparameter = quantus.ModelParameterRandomisation(
    similarity_func=quantus.similarity_func.correlation_spearman,
    return_sample_correlation=True,
    layer_order="independent",
    disable_warnings=True,
    normalise=True,
    abs=True,
    display_progressbar=True,
)

In [160]:
score_sparsity = quantus.Sparseness(return_aggregate=True, aggregate_func=np.mean, disable_warnings=True)

In [161]:
score_complexity = quantus.Complexity(
    normalise=True,
    abs=True,
    disable_warnings=True,
    return_aggregate=True
)

In [162]:
score_faithful=quantus.FaithfulnessEstimate(
    perturb_func=quantus.perturb_func.baseline_replacement_by_blur,
    similarity_func=quantus.similarity_func.correlation_pearson,
    features_in_step=1,  
    perturb_baseline="black",
    return_aggregate=True, aggregate_func=np.mean, disable_warnings=True
)

In [163]:
score_mono= quantus.MonotonicityCorrelation(
    nr_samples=5,
    features_in_step=1,
    perturb_baseline="uniform",
    perturb_func=quantus.perturb_func.gaussian_noise,
    similarity_func=quantus.similarity_func.correlation_spearman,
    return_aggregate=True, aggregate_func=np.mean, disable_warnings=True
)

In [171]:
score_robust = quantus.LocalLipschitzEstimate(
    nr_samples=5,
    perturb_std=0.02,
    perturb_mean=0.0,
    norm_numerator=quantus.similarity_func.distance_euclidean,
    norm_denominator=quantus.similarity_func.distance_euclidean,    
    perturb_func=quantus.perturb_func.gaussian_noise,
    similarity_func=quantus.similarity_func.lipschitz_constant,
    return_aggregate=True,
    return_nan_when_prediction_changes=True,
    disable_warnings=True, 
    aggregate_func=np.mean,)

In [166]:
sensitive= quantus.MaxSensitivity(
    nr_samples=10,
    lower_bound=0.2,
    norm_numerator=quantus.norm_func.fro_norm,
    norm_denominator=quantus.norm_func.fro_norm,
    perturb_func=quantus.perturb_func.uniform_noise,
    similarity_func=quantus.similarity_func.difference,
     return_aggregate=True, 
    aggregate_func=np.mean,disable_warnings=True,
    return_nan_when_prediction_changes=True,
)

In [167]:
output_stability = quantus.RelativeOutputStability(nr_samples=10, normalise=True, disable_warnings=True,)


# Compute all declared metrices for test samples and aggregate the result

In [66]:

f=[]
for i in range(1,401):
    s = score_sparsity(
        model = net, x_batch = test_input_tensor[i:i+1].cpu().numpy(), y_batch = test_labels_tensor[i:i+1].cpu().numpy(),
        a_batch = None, 
        explain_func=lime_explainer,
        explain_func_kwargs={"lime_attr": lime_attr})
    
    f.extend(s)

print(f)
print(sum(f)/len(f))


[0.41547325947943425, 0.32118323035406754, 0.3900541720034191, 0.44608237547008744, 0.37916526752599994, 0.3807433831009003, 0.3849339073443721, 0.4016042324718736, 0.40566277948061896, 0.3713605427889681, 0.4571483646029121, 0.4109763797992419, 0.39655298513095455, 0.3768432122765434, 0.4121846485063876, 0.41857065020876916, 0.3977063526389821, 0.4182182945959646, 0.40996689538960607, 0.4108300910533464, 0.41463138834333674, 0.3976839201438055, 0.41160514158336975, 0.40165084460021777, 0.412383052798366, 0.41853072413475506, 0.4270448809000387, 0.4134804265326601, 0.401426835988813, 0.4238640859123184, 0.38746999920255115, 0.4148392466059726, 0.3979435932674091, 0.41645312404500595, 0.46443602741436585, 0.42842708525753914, 0.43095300642559364, 0.4289132210208449, 0.403982719141155, 0.36985319820626616, 0.4486726916106695, 0.45712088203889367, 0.395783914384273, 0.4083431279402701, 0.4106678151897494, 0.43506427160114564, 0.3781508037078205, 0.4060148589532216, 0.3968220352019994, 0.4

In [79]:
f = []
for i in range(1,401):
    s = score_faithful(
        model = net, x_batch = test_input_tensor[i:i+1].cpu().numpy(), y_batch = test_labels_tensor[i:i+1].cpu().numpy(),
        a_batch = None, 
        explain_func=lime_explainer,
        explain_func_kwargs={"lime_attr": lime_attr})
    
    f.extend(s)

In [80]:
#print(f)
print("Faithfulness correlation estimate is ", sum(f)/len(f))

Faithfulness correlation estimate is  0.0011168948372866597


In [81]:
f = []
for i in range(1,401):
    s = score_mono(
        model = net, x_batch = test_input_tensor[i:i+1].cpu().numpy(), y_batch = test_labels_tensor[i:i+1].cpu().numpy(),
        a_batch = None, 
        explain_func=lime_explainer,
        explain_func_kwargs={"lime_attr": lime_attr})
    
    f.extend(s)

In [82]:
#print(f)
print("Monotonicity score is: ", sum(f)/len(f))

Monotonicity score is:  0.0009135497369590277


In [172]:
f = []
for i in range(1,401):
    if i%100==0:
        print(i)
        
    s = score_robust(
        model = net, x_batch = test_input_tensor[i:i+1].cpu().numpy(), y_batch = test_labels_tensor[i:i+1].cpu().numpy(),
        a_batch = None, 
        explain_func=lime_explainer,
        explain_func_kwargs={"lime_attr": lime_attr})
    
    f.extend(s)

100
200
300
400


In [130]:
#print(f)
print("robustness score is: ", sum(f)/len(f))

robustness score is:  28.343625179442153


In [169]:
f = []
for i in range(1,401):
    if i%10==0:
        print(i)
    s = sensitive(
        model = net, x_batch = test_input_tensor[i:i+1].cpu().numpy(), y_batch = test_labels_tensor[i:i+1].cpu().numpy(),
        a_batch = None, 
        explain_func=lime_explainer,
        explain_func_kwargs={"lime_attr": lime_attr})
    
    f.extend(s)

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400


In [170]:
print(f)
print("sensitivity score is: ", sum(f)/len(f))

[0.0, 0.0, 0.0, nan, nan, 0.0, 0.0, 0.0, 0.0, 0.0, nan, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, nan, 0.0, 0.0, nan, 0.0, 0.0, 0.0, 0.0, nan, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, nan, 0.0, 0.0, nan, 0.0, 0.0, 0.0, nan, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, nan, 0.0, 0.0, nan, 0.0, nan, 0.0, nan, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, nan, 0.0, nan, 0.0, nan, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, nan, 0.0, nan, nan, nan, 0.0, 0.0, 0.0, 0.0, 0.0, nan, 0.0, nan, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, nan, 0.0, nan, 0.0, 0.0, nan, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, nan, 0.0, nan, 0.0, nan, 0.0, nan, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, nan, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, nan, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, nan, 0.0, 0.0, nan, 0.0, 0.0, 0.0, 0.0, nan, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, nan, 0.0, 0.0, 0.0, 0.0, 0.0, nan, nan, nan, 0.0, nan, 0.0, nan, 0.0, 0.0, nan, 0.0, nan, nan, 0.0, 0.0, nan, 0.0, 0.0, 0.0, 0.0, nan, nan, nan, nan,

In [115]:
f = []
for i in range(1,400):
    
    s = scores_modelparameter(
        model = net, x_batch = test_input_tensor[i:i+1].cpu().numpy(), y_batch = test_labels_tensor[i:i+1].cpu().numpy(),
        a_batch = None, 
        explain_func=lime_explainer,
        explain_func_kwargs={"lime_attr": lime_attr})
    
    f.extend(s)

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

...

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

In [117]:
#print(f)
#f=np.array(f)
print("model parameter randomisation :",  sum(f)/len(f))

model parameter randomisation : 0.03198270178601845


In [111]:


f=[]
for i in range(1,401):
    
    s = score_complexity(
        model = net, x_batch = test_input_tensor[i:i+1].cpu().numpy(), y_batch = test_labels_tensor[i:i+1].cpu().numpy(),
        a_batch = None, 
        explain_func=lime_explainer,
        explain_func_kwargs={"lime_attr": lime_attr})
    
    f.extend(s)

In [113]:
#print(f)
print("Score complexity: ", sum(f)/len(f))

Score complexity:  4.622201077555321


In [107]:
f=[]
for i in range(1,100):
    if i%10==0:
        print(i)
    s = output_stability(
        model = net, x_batch = test_input_tensor[i:i+1].cpu().numpy(), y_batch = test_labels_tensor[i:i+1].cpu().numpy(),
        a_batch = None, 
        explain_func=lime_explainer,
        explain_func_kwargs={"lime_attr": lime_attr})
    
    f.extend(s)

10
20
30
40
50
60
70
80
90


In [108]:
from numpy import nanmean


f=np.array(f)
print("output stability:", nanmean(np.log(f)))

output stability: 18.0623511074426
